# IGDB
In this notebook, we get all the games' information listed in IGDB.

In [1]:
import requests
import json
import time
import pickle
import pandas as pd
import numpy as np
import os
from igdb.wrapper import IGDBWrapper

In [2]:
post_url = "https://id.twitch.tv/oauth2/token?client_id=4njc2nod9cdh7wgc575ymsij8x7pli&client_secret=2g238v7i8bk5rrch8wn3g3eg45mcgq&grant_type=client_credentials"

req = requests.post(post_url)
req_dict = req.json()

wrapper = IGDBWrapper("4njc2nod9cdh7wgc575ymsij8x7pli",req_dict.get('access_token'))

In [3]:
i=0
games_list = []
while True:
    byte_array = wrapper.api_request(
            'games',
            'fields age_ratings,aggregated_rating,aggregated_rating_count,alternative_names,artworks,bundles,category,checksum,collection,cover,created_at,dlcs,expanded_games,expansions,external_games,first_release_date,follows,forks,franchise,franchises,game_engines,game_localizations,game_modes,genres,hypes,involved_companies,keywords,language_supports,multiplayer_modes,name,parent_game,platforms,player_perspectives,ports,rating,rating_count,release_dates,remakes,remasters,screenshots,similar_games,slug,standalone_expansions,status,storyline,summary,tags,themes,total_rating,total_rating_count,updated_at,url,version_parent,version_title,videos,websites; offset %d; limit 500;'%(i*500)
          )
    i+=1
    games = json.loads(byte_array)
    games_list.extend(games)
    print(i)
    time.sleep(0.2)
    if games==[]:
        break

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [4]:
if not os.path.exists('./data/games.pkl'):
    with open('./data/games.pkl','wb') as f:
        pickle.dump(games_list, f)

In [2]:
with open('./data/games.pkl','rb') as f:
    games_list = pickle.load(f)

In [6]:
# IGDB contains 230,222 games.
df = pd.DataFrame.from_dict(games_list)
df.shape

(230222, 57)

In [7]:
df.head(5)

,id,category,created_at,external_games,first_release_date,game_modes,genres,name,platforms,release_dates,...,franchises,franchise,multiplayer_modes,dlcs,expansions,ports,remakes,expanded_games,standalone_expansions,forks
0,35004,0,1495818975,"[8077, 1957344]",1.437696e+09,"[2, 5]","[15, 32]",Demon Horde Master,[6],[80453],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,151599,0,1622835085,"[2060085, 2077186, 2593975, 2637093]",1.623888e+09,[1],"[31, 32, 34]",Idol Days,"[6, 130]","[251428, 338327, 373398]",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,244927,0,1680957248,"[2690306, 2691475]",NaN,NaN,"[15, 32]",Wave of Elms,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,118008,0,1557063612,"[1720438, 1864275, 1918495]",1.589501e+09,"[1, 2, 3]","[13, 15, 32]",Tabletop Playground,[6],[220820],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,95080,0,1521818623,[1989881],NaN,NaN,NaN,Dotra,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
# Load the platforms
with open('./data/platforms.pkl', 'rb') as f:
    platforms = pickle.load(f)
    
# To dateframe
df = pd.DataFrame(platforms)
# Transform release_dates to datetime
for platform in platforms:
    try:
        platform['release_dates'] = pd.to_datetime(platform['release_dates'], unit='s')
    except:
        pass

# Choose the platforms that are released after 2005
platforms = [platform for platform in platforms if platform['release_dates'] and platform['release_dates'].get('y')>=2009]
# Convert the platforms to dataframe
df_platforms = pd.DataFrame(platforms)

In [23]:
df_platforms

,id,alternative_name,category,created_at,name,platform_logo,slug,updated_at,url,versions,websites,checksum,release_dates,abbreviation,generation,platform_family,summary
0,158,Commodore Dynamic Total Vision,6.0,1510012800,Commodore CDTV,292.0,commodore-cdtv,1522972800,https://www.igdb.com/platforms/commodore-cdtv,[223],[26],5d48648e-283c-e2df-df7d-6d3cbc7e0a58,"{'id': 158, 'category': 0, 'date': 1382400000,...",NaN,NaN,NaN,NaN
1,237,NaN,6.0,1573776000,Sol-20,NaN,sol-20,1591228800,https://www.igdb.com/platforms/sol-20,[354],[219],c608e02f-1a92-d010-b161-4a368220bdfe,"{'id': 237, 'category': 0, 'date': 1271289600,...",NaN,NaN,NaN,NaN
2,6,mswin,4.0,1297555200,PC (Microsoft Windows),203.0,win,1470009600,https://www.igdb.com/platforms/win,"[1, 13, 14, 15, 124]",[2],5aae54d0-390e-a4ec-a9ee-4ad4cc346992,"{'id': 6, 'category': 0, 'date': 1351209600, '...",PC,NaN,NaN,NaN
3,156,NaN,6.0,1510012800,Thomson MO5,537.0,thomson-mo5,1510012800,https://www.igdb.com/platforms/thomson-mo5,[221],[24],5f4cdd4c-4bc0-2a7d-9842-5d1c5350ab4f,"{'id': 156, 'category': 0, 'date': 1311120000,...",NaN,NaN,NaN,NaN
4,163,NaN,NaN,1518652800,SteamVR,159.0,steam-vr,1561939200,https://www.igdb.com/platforms/steam-vr,[233],[30],e2c8f5f7-2c50-5d2d-b9b9-1135281d38f4,"{'id': 163, 'category': 0, 'date': 1348790400,...",Steam VR,NaN,NaN,NaN
5,149,NaN,6.0,1508976000,PC-98,366.0,pc-98,1508976000,https://www.igdb.com/platforms/pc-98,[214],NaN,da17ec85-249b-d4ca-f876-8376319ba319,"{'id': 149, 'category': 0, 'date': 1276732800,...",NaN,NaN,NaN,NaN
6,50,3DO,1.0,1326829565,3DO Interactive Multiplayer,282.0,3do,1633117171,https://www.igdb.com/platforms/3do,[18],NaN,87213d2d-1406-71e3-dc09-8f2352937191,"{'id': 50, 'category': 0, 'date': 1399075200, ...",3DO,5.0,NaN,NaN
7,65,NaN,6.0,1372464000,Atari 8-bit,373.0,atari8bit,1468540800,https://www.igdb.com/platforms/atari8bit,"[27, 104]",NaN,37adef88-b15f-3963-a417-687e5e0a6260,"{'id': 65, 'category': 0, 'date': 1287619200, ...",Atari8bit,NaN,NaN,NaN
8,13,PC DOS,4.0,1297672223,DOS,38.0,dos,1656450592,https://www.igdb.com/platforms/dos,"[56, 540]",NaN,9aabba9b-eb06-5274-26e1-250114855890,"{'id': 13, 'category': 0, 'date': 1385683200, ...",DOS,NaN,NaN,NaN
9,155,NaN,6.0,1510012800,Tatung Einstein,368.0,tatung-einstein,1510012800,https://www.igdb.com/platforms/tatung-einstein,[220],[23],e7061776-b061-c4e7-b04e-cac5d68c38d1,"{'id': 155, 'category': 0, 'date': 1251417600,...",NaN,NaN,NaN,NaN
